In [3]:
import sys
import os

# Ensure the module path is correct
src_path = r'C:\Users\Sami\Desktop\Uni\Flow-EEG-Psych\src'
if src_path not in sys.path:
    sys.path.append(src_path)

from features.extract_features import main as extract_features_main

# Run the feature extraction
extract_features_main()



Error importing the feature extraction function: cannot import name 'main' from 'features.extract_features' (C:\Users\Sami\Desktop\Uni\Flow-EEG-Psych\src\features\extract_features.py)
